<a href="https://colab.research.google.com/github/eddiechn/Multi-Class-Animal-Classification/blob/main/Multi_Animal_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries - data science
import numpy as np
import pandas as pd
import tensorflow as tf
import random


# import libraries - visualization
import matplotlib.pyplot as plt

# import libraries - system
import os
from pathlib import Path


In [ ]:
!pip install opendatasets

In [3]:
# seeding everyting to reproduce results for future cases
def seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed()


In [4]:
# data
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [5]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/alessiocorrado99/animals10/data?select=raw-img')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: eddiechn1
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10


100%|██████████| 586M/586M [00:03<00:00, 164MB/s]


In [7]:
from translate import translate
print(translate)

{'cane': 'dog', 'cavallo': 'horse', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'pecora': 'sheep', 'scoiattolo': 'squirrel', 'dog': 'cane', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'spider': 'ragno', 'squirrel': 'scoiattolo'}


In [9]:
dataset_dir = 'animals10/raw-img'

In [16]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    batch_size = BATCH_SIZE,
    image_size = IMG_SIZE,
    label_mode='int'
)

Found 26179 files belonging to 10 classes.


In [17]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
from PIL import Image
